In [1]:
import random
import numpy as np
import torch
import os

def set_seed(seed):
    """
    设置所有随机种子以确保结果可复现
    
    Args:
        seed (int): 随机种子数值
    """
    random.seed(seed)  # Python的随机种子
    np.random.seed(seed)  # Numpy的随机种子
    torch.manual_seed(seed)  # PyTorch的CPU随机种子
    torch.cuda.manual_seed(seed)  # PyTorch的GPU随机种子
    torch.cuda.manual_seed_all(seed)  # 如果使用多GPU，为所有GPU设置种子
    
    # 设置cudnn的随机种子
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # 设置Python的hash种子
    os.environ['PYTHONHASHSEED'] = str(seed)
seed = 2003  # 可以设置任何整数
set_seed(seed)

In [2]:
import os
import shutil
from tqdm import tqdm

def organize_files(src_root_dir, dst_root_dir):
    cases = ['case1', 'case2', 'case3', 'case4']

    # Create directories for each case
    for case in cases:
        case_dir = os.path.join(dst_root_dir, case)
        if not os.path.exists(case_dir):
            os.makedirs(case_dir)

    # Traverse the source directory
    for main_dir in tqdm(os.listdir(src_root_dir), desc="Processing directories"):
        main_dir_path = os.path.join(src_root_dir, main_dir)
        if os.path.isdir(main_dir_path):
            for sub_dir in os.listdir(main_dir_path):
                sub_dir_path = os.path.join(main_dir_path, sub_dir)
                if os.path.isdir(sub_dir_path):
                    case_name = sub_dir.split('_')[0]
                    case_dir = os.path.join(dst_root_dir, case_name)

                    for filename in os.listdir(sub_dir_path):
                        if filename.endswith('.wav'):
                            src_wav_path = os.path.join(sub_dir_path, filename)
                            dst_wav_path = os.path.join(case_dir, f"{main_dir}_{sub_dir}_{filename}")
                            shutil.copy(src_wav_path, dst_wav_path)
                            # print(f"Copied {src_wav_path} to {dst_wav_path}")

                            # Copy corresponding .npy file
                            src_npy_path = src_wav_path.replace('.wav', '.npy')
                            dst_npy_path = dst_wav_path.replace('.wav', '.npy')
                            if os.path.exists(src_npy_path):
                                shutil.copy(src_npy_path, dst_npy_path)
                                # print(f"Copied {src_npy_path} to {dst_npy_path}")
src_directory = 'dataset_2k3k_withbandpass_extrafeatures'
dst_directory = 'data_2k3k_nobandpass_organized_dataset_extrafeatures'
    
print(f"源目录: {src_directory}")
print(f"目标目录: {dst_directory}")

if not os.path.exists(src_directory):
    print(f"源目录不存在: {src_directory}")
    
organize_files(src_directory, dst_directory)
print("文件整理完成。")

源目录: dataset_2k3k_withbandpass_extrafeatures
目标目录: data_2k3k_nobandpass_organized_dataset_extrafeatures


Processing directories: 100%|██████████| 19/19 [00:02<00:00,  9.48it/s]

文件整理完成。


In [3]:
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm import tqdm

import torchaudio
from sklearn.model_selection import train_test_split
import os
import sys

In [4]:
import os
from pathlib import Path
import torchaudio
import numpy as np
from tqdm import tqdm

data = []

for case in ['case1', 'case2', 'case3', 'case4']:
    case_path = Path(f'data_2k3k_nobandpass_organized_dataset_extrafeatures/{case}')
    for path in tqdm(case_path.glob("*.wav"), desc=f"Processing {case}"):
        name = path.stem
        # 获取文件名的各个部分
        parts = path.name.split('_')
        prefix = parts[0]  # 前缀
        case_id = '_'.join(parts[1:-2])  # case_id（可能包含多个部分）
        sample_set = parts[-2]  # sample_set

        try:
            # 加载音频文件
            audio, sr = torchaudio.load(path)
            
            # 加载对应的 .npy 文件
            npy_path = path.with_suffix('.npy')
            if npy_path.exists():
                energy_features = np.load(npy_path)
                energy_features = energy_features[0]
            else:
                energy_features = None
                print(f"Warning: No .npy file found for {path}")

            data.append({
                "name": name,
                "path": str(path),
                "case": case,
                "prefix": prefix,
                "case_id": case_id,
                "sample_set": sample_set,
                "audio": audio,
                "sr": sr,
                "energy_features": energy_features
            })
        except Exception as e:
            print(f"Error processing file {path}: {str(e)}")
            # 跳过损坏的文件
            pass

# 显示收集到的数据条目数
print(f"Collected {len(data)} items.")

# 数据统计
case_counts = {case: sum(1 for item in data if item['case'] == case) for case in ['case1', 'case2', 'case3', 'case4']}
print("\nData distribution:")
for case, count in case_counts.items():
    print(f"{case}: {count} items")

# 检查是否所有项目都有能量特征
items_with_features = sum(1 for item in data if item['energy_features'] is not None)
print(f"\nItems with energy features: {items_with_features} out of {len(data)}")

Processing case1: 456it [00:03, 120.45it/s]
Processing case2: 456it [00:05, 86.30it/s]
Processing case3: 912it [00:09, 95.18it/s] 
Processing case4: 912it [00:09, 99.63it/s] 

Collected 2736 items.

Data distribution:
case1: 456 items
case2: 456 items
case3: 912 items
case4: 912 items

Items with energy features: 2736 out of 2736


In [5]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,name,path,case,prefix,case_id,sample_set,audio,sr,energy_features
0,A10_case1_1_sample_10_1,data_2k3k_nobandpass_organized_dataset_extrafe...,case1,A10,case1_1_sample,10,"[[tensor(0.0045), tensor(0.0026), tensor(0.004...",44100,1.002035
1,A10_case1_1_sample_11_2,data_2k3k_nobandpass_organized_dataset_extrafe...,case1,A10,case1_1_sample,11,"[[tensor(-0.0096), tensor(-0.0103), tensor(-0....",44100,0.000004
2,A10_case1_1_sample_13_1,data_2k3k_nobandpass_organized_dataset_extrafe...,case1,A10,case1_1_sample,13,"[[tensor(-0.0014), tensor(-0.0056), tensor(-0....",44100,1.000000
3,A10_case1_1_sample_14_2,data_2k3k_nobandpass_organized_dataset_extrafe...,case1,A10,case1_1_sample,14,"[[tensor(0.0051), tensor(0.0046), tensor(0.006...",44100,0.000005
4,A10_case1_1_sample_16_1,data_2k3k_nobandpass_organized_dataset_extrafe...,case1,A10,case1_1_sample,16,"[[tensor(0.0045), tensor(0.0026), tensor(0.004...",44100,1.002035


In [6]:
import pandas as pd
df = pd.DataFrame(data)
df.head()
df = df.sample(frac=1).reset_index(drop=True)

Let's display some random sample of the dataset and run it a couple of times to get a feeling for the audio and the emotional label.

In [7]:
import os
import pandas as pd
import torchaudio
import librosa
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from pathlib import Path
from datasets import load_dataset, Dataset

save_path = "data_2k3k_nobandpass_organized_dataset_extrafeatures"

# 确保 df 中包含 energy_features_path
df['energy_features_path'] = df['path'].apply(lambda x: x.replace('.wav', '.npy'))

# 使用 "case" 标签进行 stratify
train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["case"])

# 重置索引
train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)

# 保存为 CSV 文件
os.makedirs(save_path, exist_ok=True)
train_df.to_csv(f"{save_path}/train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv(f"{save_path}/test.csv", sep="\t", encoding="utf-8", index=False)

# 打印数据集的形状
print("Train dataset shape:", train_df.shape)
print("Test dataset shape:", test_df.shape)

# 加载数据集
data_files = {
    "train": f"{save_path}/train.csv",
    "validation": f"{save_path}/test.csv",
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

# 指定输入和输出列
input_column = "path"
output_column = "case"

# 打印数据集的前几行，检查数据完整性
print("Train dataset preview:")
print(train_dataset[:5])
print("Validation dataset preview:")
print(eval_dataset[:5])

# 打印每个 case 的样本数
print("Sample count per case in training dataset:")
print(train_dataset.to_pandas()[output_column].value_counts())
print("Sample count per case in validation dataset:")
print(eval_dataset.to_pandas()[output_column].value_counts())

# 识别和排序标签列表
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

# 加载能量特征
def load_energy_features(example):
    energy_path = example.get('energy_features_path')
    if energy_path and os.path.exists(energy_path):
        try:
            example['energy_features'] = np.load(energy_path).tolist()
        except Exception as e:
            print(f"Error loading energy features from {energy_path}: {e}")
            example['energy_features'] = None
    else:
        example['energy_features'] = None
    return example

train_dataset = train_dataset.map(load_energy_features)
eval_dataset = eval_dataset.map(load_energy_features)

# 检查能量特征加载情况
print("Train dataset with energy features:")
print(train_dataset[:5])
print("Validation dataset with energy features:")
print(eval_dataset[:5])

# 统计包含能量特征的样本数
train_with_features = sum(1 for item in train_dataset if item['energy_features'] is not None)
eval_with_features = sum(1 for item in eval_dataset if item['energy_features'] is not None)
print(f"Train samples with energy features: {train_with_features} out of {len(train_dataset)}")
print(f"Validation samples with energy features: {eval_with_features} out of {len(eval_dataset)}")

c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train dataset shape: (2188, 10)
Test dataset shape: (548, 10)


Generating train split: 2188 examples [00:00, 140130.36 examples/s]
Generating validation split: 548 examples [00:00, 106121.18 examples/s]


Train dataset preview:
{'name': ['E9_case3_8_sample_47_2', 'E1_case3_7_sample_38_1', 'E11_case2_4_sample_21_2', 'E6_case1_1_sample_14_2', 'A9_case3_7_sample_40_1'], 'path': ['data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\E9_case3_8_sample_47_2.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\E1_case3_7_sample_38_1.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case2\\E11_case2_4_sample_21_2.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case1\\E6_case1_1_sample_14_2.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\A9_case3_7_sample_40_1.wav'], 'case': ['case3', 'case3', 'case2', 'case1', 'case3'], 'prefix': ['E9', 'E1', 'E11', 'E6', 'A9'], 'case_id': ['case3_8_sample', 'case3_7_sample', 'case2_4_sample', 'case1_1_sample', 'case3_7_sample'], 'sample_set': [47, 38, 21, 14, 40], 'audio': ['tensor([[ 0.0965,  0.0119, -0.0739,  ..., -0.1732, -0.1497, -0.1315]])', 'tensor([[ 0.0767,  0.0316, -0.0219,  ..., -0.10

Map: 100%|██████████| 548/548 [00:00<00:00, 3390.08 examples/s]


Train dataset with energy features:
{'name': ['E9_case3_8_sample_47_2', 'E1_case3_7_sample_38_1', 'E11_case2_4_sample_21_2', 'E6_case1_1_sample_14_2', 'A9_case3_7_sample_40_1'], 'path': ['data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\E9_case3_8_sample_47_2.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\E1_case3_7_sample_38_1.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case2\\E11_case2_4_sample_21_2.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case1\\E6_case1_1_sample_14_2.wav', 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\A9_case3_7_sample_40_1.wav'], 'case': ['case3', 'case3', 'case2', 'case1', 'case3'], 'prefix': ['E9', 'E1', 'E11', 'E6', 'A9'], 'case_id': ['case3_8_sample', 'case3_7_sample', 'case2_4_sample', 'case1_1_sample', 'case3_7_sample'], 'sample_set': [47, 38, 21, 14, 40], 'audio': ['tensor([[ 0.0965,  0.0119, -0.0739,  ..., -0.1732, -0.1497, -0.1315]])', 'tensor([[ 0.0767,  0.0316, -0.0219

## Prepare Data for Training

In [8]:
# Loading the created dataset using datasets
from datasets import load_dataset

# 定义数据文件路径
data_files = {
    "train": "data_2k3k_nobandpass_organized_dataset_extrafeatures/train.csv",
    "validation": "data_2k3k_nobandpass_organized_dataset_extrafeatures/test.csv",
}

# 加载数据集
dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

# 打印数据集信息
print(train_dataset)
print(eval_dataset)


Dataset({
    features: ['name', 'path', 'case', 'prefix', 'case_id', 'sample_set', 'audio', 'sr', 'energy_features', 'energy_features_path'],
    num_rows: 2188
})
Dataset({
    features: ['name', 'path', 'case', 'prefix', 'case_id', 'sample_set', 'audio', 'sr', 'energy_features', 'energy_features_path'],
    num_rows: 548
})


In [9]:
input_column = "path"
output_column = "case"

In [10]:
# we need to distinguish the unique labels in our SER dataset
label_list = train_dataset.unique(output_column)
label_list.sort()  # Let's sort it for determinism
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

A classification problem with 4 classes: ['case1', 'case2', 'case3', 'case4']


In order to preprocess the audio into our classification model, we need to set up the relevant Wav2Vec2 assets regarding our language in this case `lighteternal/wav2vec2-large-xlsr-53-greek` fine-tuned by [Dimitris Papadopoulos](https://huggingface.co/lighteternal/wav2vec2-large-xlsr-53-greek). To handle the context representations in any audio length we use a merge strategy plan (pooling mode) to concatenate that 3D representations into 2D representations.

There are three merge strategies `mean`, `sum`, and `max`. In this example, we achieved better results on the mean approach. In the following, we need to initiate the config and the feature extractor from the Dimitris model.

In [11]:
from transformers import AutoConfig, Wav2Vec2Processor

In [12]:
model_name_or_path = "c3f9d884181a224a6ac87bf8885c84d1cff3384f"
pooling_mode = "mean"

In [13]:
# config
config = AutoConfig.from_pretrained(
    model_name_or_path,
    num_labels=num_labels,
    label2id={label: i for i, label in enumerate(label_list)},
    id2label={i: label for i, label in enumerate(label_list)},
    finetuning_task="audio_classification",
)
setattr(config, 'pooling_mode', pooling_mode)

setattr(config, 'use_energy_features', True)
setattr(config, 'energy_feature_dim', 1)

In [14]:
# processor = Wav2Vec2Processor.from_pretrained(model_name_or_path,)
from transformers import Wav2Vec2FeatureExtractor
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)
# target_sampling_rate = processor.feature_extractor.sampling_rate
target_sampling_rate = 16000
print(f"The target sampling rate: {target_sampling_rate}")

The target sampling rate: 16000


# Preprocess Data

So far, we downloaded, loaded, and split the SER dataset into train and test sets. The instantiated our strategy configuration for using context representations in our classification problem SER. Now, we need to extract features from the audio path in context representation tensors and feed them into our classification model to determine the emotion in the speech.

Since the audio file is saved in the `.wav` format, it is easy to use **[Librosa](https://librosa.org/doc/latest/index.html)** or others, but we suppose that the format may be in the `.mp3` format in case of generality. We found that the **[Torchaudio](https://pytorch.org/audio/stable/index.html)** library works best for reading in `.mp3` data.

An audio file usually stores both its values and the sampling rate with which the speech signal was digitalized. We want to store both in the dataset and write a **map(...)** function accordingly. Also, we need to handle the string labels into integers for our specific classification task in this case, the **single-label classification** you may want to use for your **regression** or even **multi-label classification**.

In [15]:
import torchaudio
import numpy as np

def speech_file_to_array_fn(path, target_sampling_rate=16000):
    speech_array, sampling_rate = torchaudio.load(path)
    if sampling_rate != target_sampling_rate:
        resampler = torchaudio.transforms.Resample(sampling_rate, target_sampling_rate)
        speech_array = resampler(speech_array)
    speech = speech_array.squeeze().numpy()
    return speech

def label_to_id(label, label_list):
    if isinstance(label_list, list) and len(label_list) > 0:
        return label_list.index(label) if label in label_list else -1
    return label

# 新增函数：加载能量特征
def load_energy_features(path):
    try:
        return np.load(path)
    except Exception as e:
        print(f"Error loading energy features from {path}: {e}")
        return None

In [16]:
from transformers import Wav2Vec2FeatureExtractor
from datasets import load_dataset
import torchaudio
import numpy as np

# 确保 model_name_or_path 已定义
model_name_or_path = "c3f9d884181a224a6ac87bf8885c84d1cff3384f"
processor = Wav2Vec2FeatureExtractor.from_pretrained(model_name_or_path)

# 定义输入和输出列
input_column = "path"
output_column = "case"

# 定义预处理函数
def preprocess_function(examples):
    audio = [speech_file_to_array_fn(path) for path in examples["path"]]
    result = processor(audio, sampling_rate=16000, return_tensors="pt", padding=True)
    result["labels"] = [label_to_id(label, label_list) for label in examples["case"]]
    
    # 加载能量特征
    energy_features = []
    for path in examples["energy_features_path"]:
        if path:
            features = load_energy_features(path)
            if features is not None:
                features = np.array(features)
            energy_features.append(features)
        else:
            energy_features.append(None)
            
    result["energy_features"] = energy_features
    
    return result

# 加载数据集
data_files = {
    "train": "data_2k3k_nobandpass_organized_dataset_extrafeatures/train.csv",
    "validation": "data_2k3k_nobandpass_organized_dataset_extrafeatures/test.csv",
}
dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]

# 获取独特的标签
label_list = train_dataset.unique("case")
label_list.sort()  # 排序以确保确定性
num_labels = len(label_list)
print(f"A classification problem with {num_labels} classes: {label_list}")

# 预处理和映射数据集
train_dataset = train_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=1,
    remove_columns=["name", "prefix", "case_id", "sample_set"]
)
eval_dataset = eval_dataset.map(
    preprocess_function,
    batch_size=100,
    batched=True,
    num_proc=1,
    remove_columns=["name", "prefix", "case_id", "sample_set"]
)

print(train_dataset)
print(eval_dataset)

# 检查处理后的数据集
print("\nProcessed train dataset example:")
print(train_dataset[0])
print("\nProcessed eval dataset example:")
print(eval_dataset[0])

# 统计包含能量特征的样本数
train_with_features = sum(1 for item in train_dataset if item['energy_features'] is not None)
eval_with_features = sum(1 for item in eval_dataset if item['energy_features'] is not None)
print(f"\nTrain samples with energy features: {train_with_features} out of {len(train_dataset)}")
print(f"Validation samples with energy features: {eval_with_features} out of {len(eval_dataset)}")

A classification problem with 4 classes: ['case1', 'case2', 'case3', 'case4']


Map: 100%|██████████| 548/548 [00:01<00:00, 516.87 examples/s]


Dataset({
    features: ['path', 'case', 'audio', 'sr', 'energy_features', 'energy_features_path', 'input_values', 'attention_mask', 'labels'],
    num_rows: 2188
})
Dataset({
    features: ['path', 'case', 'audio', 'sr', 'energy_features', 'energy_features_path', 'input_values', 'attention_mask', 'labels'],
    num_rows: 548
})

Processed train dataset example:
{'path': 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\E9_case3_8_sample_47_2.wav', 'case': 'case3', 'audio': 'tensor([[ 0.0965,  0.0119, -0.0739,  ..., -0.1732, -0.1497, -0.1315]])', 'sr': 44100, 'energy_features': [1.017976454971555], 'energy_features_path': 'data_2k3k_nobandpass_organized_dataset_extrafeatures\\case3\\E9_case3_8_sample_47_2.npy', 'input_values': [0.30043041706085205, -0.8148924112319946, -1.348052740097046, -0.435878723859787, 0.5519919991493225, 0.6039494872093201, -0.32139089703559875, -0.8966872096061707, -0.3445829153060913, 0.6257559061050415, 0.5910144448280334, -0.541273832321167, -1.0

In [17]:
idx = 4
print(f"Training input_values: {train_dataset[idx]['input_values']}")
print(f"Training attention_mask: {train_dataset[idx]['attention_mask']}")
print(f"Training labels: {train_dataset[idx]['labels']} - {train_dataset[idx]['case']}")

Training input_values: [-0.1631801277399063, -0.08776722103357315, 0.16888834536075592, 0.10686714202165604, -0.2315858006477356, -0.3351719081401825, -0.14892247319221497, -0.06371539831161499, -0.1644662469625473, -0.17130061984062195, 0.10301103442907333, 0.26819083094596863, 0.08521825820207596, -0.14896169304847717, -0.3138553202152252, -0.07379782199859619, 0.2684226334095001, 0.10978315025568008, -0.04995983839035034, 0.3490457236766815, 0.6290960907936096, 0.02939767576754093, -0.4464440941810608, -0.08138640969991684, 0.1198454275727272, -0.10316295921802521, -0.010972526855766773, 0.5891613960266113, 0.6200134754180908, -0.5227492451667786, -1.0758397579193115, -0.3097827434539795, 0.3039565682411194, 0.0937492847442627, -0.32498541474342346, -0.23230832815170288, 0.12762382626533508, -0.013073979876935482, -0.30112239718437195, -0.229781374335289, -0.25356176495552063, -0.22165115177631378, 0.2635350525379181, 0.49902480840682983, 0.07101009786128998, -0.48568007349967957, -

Great, now we've successfully read all the audio files, resampled the audio files to 16kHz, and mapped each audio to the corresponding label.

## Model

Before diving into the training part, we need to build our classification model based on the merge strategy.

In [18]:
from dataclasses import dataclass
from typing import Optional, Tuple
import torch
from transformers.file_utils import ModelOutput


@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None


In [19]:
import torch
import torch.nn as nn
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss

from transformers.models.wav2vec2.modeling_wav2vec2 import (
    Wav2Vec2PreTrainedModel,
    Wav2Vec2Model
)

class Wav2Vec2ClassificationHead(nn.Module):
    """Head for wav2vec classification task."""

    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size + config.energy_feature_dim, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, features, energy_features=None, **kwargs):
        x = features
        if energy_features is not None:
            x = torch.cat([x, energy_features], dim=-1)
        x = self.dropout(x)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

class Wav2Vec2ForSpeechClassification(Wav2Vec2PreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config

        self.wav2vec2 = Wav2Vec2Model(config)
        self.classifier = Wav2Vec2ClassificationHead(config)

        self.init_weights()

    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()

    def merged_strategy(self, hidden_states, mode="mean"):
        if mode == "mean":
            outputs = torch.mean(hidden_states, dim=1)
        elif mode == "sum":
            outputs = torch.sum(hidden_states, dim=1)
        elif mode == "max":
            outputs = torch.max(hidden_states, dim=1)[0]
        else:
            raise Exception(
                "The pooling method hasn't been defined! Your pooling mode must be one of these ['mean', 'sum', 'max']")
        return outputs

    def forward(
            self,
            input_values,
            attention_mask=None,
            energy_features=None,
            output_attentions=None,
            output_hidden_states=None,
            return_dict=None,
            labels=None,
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        hidden_states = outputs[0]
        # print(f"Hidden states shape before: {hidden_states.shape}")
        hidden_states = self.merged_strategy(hidden_states, mode=self.pooling_mode)
        # print(f"Hidden states shape after: {hidden_states.shape}")

        logits = self.classifier(hidden_states, energy_features)

        loss = None
        if labels is not None:
            if self.config.problem_type is None:
                if self.num_labels == 1:
                    self.config.problem_type = "regression"
                elif self.num_labels > 1 and (labels.dtype == torch.long or labels.dtype == torch.int):
                    self.config.problem_type = "single_label_classification"
                else:
                    self.config.problem_type = "multi_label_classification"

            if self.config.problem_type == "regression":
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels)
            elif self.config.problem_type == "single_label_classification":
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            elif self.config.problem_type == "multi_label_classification":
                loss_fct = BCEWithLogitsLoss()
                loss = loss_fct(logits, labels)

        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SpeechClassifierOutput(
            loss=loss,
            logits=logits,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

## Training

The data is processed so that we are ready to start setting up the training pipeline. We will make use of 🤗's [Trainer](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer) for which we essentially need to do the following:

- Define a data collator. In contrast to most NLP models, XLSR-Wav2Vec2 has a much larger input length than output length. *E.g.*, a sample of input length 50000 has an output length of no more than 100. Given the large input sizes, it is much more efficient to pad the training batches dynamically meaning that all training samples should only be padded to the longest sample in their batch and not the overall longest sample. Therefore, fine-tuning XLSR-Wav2Vec2 requires a special padding data collator, which we will define below

- Evaluation metric. During training, the model should be evaluated on the word error rate. We should define a `compute_metrics` function accordingly

- Load a pretrained checkpoint. We need to load a pretrained checkpoint and configure it correctly for training.

- Define the training configuration.

After having fine-tuned the model, we will correctly evaluate it on the test data and verify that it has indeed learned to correctly transcribe speech.

### Set-up Trainer

Let's start by defining the data collator. The code for the data collator was copied from [this example](https://github.com/huggingface/transformers/blob/9a06b6b11bdfc42eea08fa91d0c737d1863c99e3/examples/research_projects/wav2vec2/run_asr.py#L81).

Without going into too many details, in contrast to the common data collators, this data collator treats the `input_values` and `labels` differently and thus applies to separate padding functions on them (again making use of XLSR-Wav2Vec2's context manager). This is necessary because in speech input and output are of different modalities meaning that they should not be treated by the same padding function.
Analogous to the common data collators, the padding tokens in the labels with `-100` so that those tokens are **not** taken into account when computing the loss.

In [20]:
from dataclasses import dataclass
from typing import Dict, List, Optional, Union
import torch
import numpy as np

import transformers
from transformers import Wav2Vec2Processor


@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for processing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor, np.ndarray]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]
        energy_features = [feature.get("energy_features") for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        if all(ef is not None for ef in energy_features):
            # 如果所有样本都有能量特征
            batch["energy_features"] = torch.tensor(np.array(energy_features), dtype=torch.float)
        else:
            print("Warning: Some samples are missing energy features. Using zero padding.")
            max_length = max(ef.shape[0] if ef is not None else 0 for ef in energy_features)
            padded_energy_features = []
            for ef in energy_features:
                if ef is not None:
                    padded_ef = np.pad(ef, ((0, max_length - ef.shape[0]), (0, 0)), mode='constant')
                padded_energy_features.append(padded_ef)
            batch["energy_features"] = torch.tensor(np.array(padded_energy_features), dtype=torch.float)

        return batch

In [21]:
from transformers import Wav2Vec2FeatureExtractor

processor = Wav2Vec2FeatureExtractor.from_pretrained('c3f9d884181a224a6ac87bf8885c84d1cff3384f')
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

config.energy_feature_dim = 1

Next, the evaluation metric is defined. There are many pre-defined metrics for classification/regression problems, but in this case, we would continue with just **Accuracy** for classification and **MSE** for regression. You can define other metrics on your own.

In [22]:
is_regression = False

In [23]:
import numpy as np
from transformers import EvalPrediction


def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.squeeze(preds) if is_regression else np.argmax(preds, axis=1)

    if is_regression:
        return {"mse": ((preds - p.label_ids) ** 2).mean().item()}
    else:
        return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

Now, we can load the pretrained XLSR-Wav2Vec2 checkpoint into our classification model with a pooling strategy.

In [24]:
model = Wav2Vec2ForSpeechClassification.from_pretrained(
    model_name_or_path,
    config=config,
)

Some weights of Wav2Vec2ForSpeechClassification were not initialized from the model checkpoint at c3f9d884181a224a6ac87bf8885c84d1cff3384f and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The first component of XLSR-Wav2Vec2 consists of a stack of CNN layers that are used to extract acoustically meaningful - but contextually independent - features from the raw speech signal. This part of the model has already been sufficiently trained during pretraining and as stated in the [paper](https://arxiv.org/pdf/2006.13979.pdf) does not need to be fine-tuned anymore.
Thus, we can set the `requires_grad` to `False` for all parameters of the *feature extraction* part.

In [25]:
model.freeze_feature_extractor()

In a final step, we define all parameters related to training.
To give more explanation on some of the parameters:
- `learning_rate` and `weight_decay` were heuristically tuned until fine-tuning has become stable. Note that those parameters strongly depend on the Common Voice dataset and might be suboptimal for other speech datasets.

For more explanations on other parameters, one can take a look at the [docs](https://huggingface.co/transformers/master/main_classes/trainer.html?highlight=trainer#trainingarguments).

**Note**: If one wants to save the trained models in his/her google drive the commented-out `output_dir` can be used instead.

In [26]:
# from google.colab import drive

# drive.mount('/gdrive')

In [27]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="train_result/audio_with_energyfeature",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=1,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    logging_steps=10,
    learning_rate=1e-4,
    save_total_limit=2,
    seed=2003,
)

c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# !pip uninstall transformers[torch] -y
# !pip install transformers[torch]

In [29]:
# !pip uninstall transformers -y
# !pip install transformers


For future use we can create our training script, we do it in a simple way. You can add more on you own.

In [30]:
from typing import Any, Dict, Union

import torch
from packaging import version
from torch import nn

from transformers import (
    Trainer,
    is_apex_available,
)

if is_apex_available():
    from apex import amp

if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast


class CTCTrainer(Trainer):
    def training_step(self, model: nn.Module, inputs: Dict[str, Union[torch.Tensor, Any]]) -> torch.Tensor:
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (:obj:`nn.Module`):
                The model to train.
            inputs (:obj:`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument :obj:`labels`. Check your model's documentation for all accepted arguments.

        Return:
            :obj:`torch.Tensor`: The tensor with training loss on this batch.
        """

        model.train()
        inputs = self._prepare_inputs(inputs)

        if self.use_cuda_amq:
            with autocast():
                loss = self.compute_loss(model, inputs)
        else:
            loss = self.compute_loss(model, inputs)

        if self.args.gradient_accumulation_steps > 1:
            loss = loss / self.args.gradient_accumulation_steps

        if self.use_cuda_amq:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        elif self.deepspeed:
            self.deepspeed.backward(loss)
        else:
            loss.backward()

        return loss.detach()


Now, all instances can be passed to Trainer and we are ready to start training!

In [31]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    # tokenizer=processor.feature_extractor,
)


### Training

Training will take between 10 and 60 minutes depending on the GPU allocated to this notebook.

In case you want to use this google colab to fine-tune your model, you should make sure that your training doesn't stop due to inactivity. A simple hack to prevent this is to paste the following code into the console of this tab (right mouse click -> inspect -> Console tab and insert code).

\\```javascript
function ConnectButton(){
    console.log("Connect pushed");
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
```

In [32]:
trainer.train()

  0%|          | 0/273 [00:00<?, ?it/s]c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,
c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\transformers\models\wav2vec2\modeling_wav2vec2.py:862: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\81951\miniconda3\envs\eeg\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECU

{'loss': 1.3238, 'grad_norm': 7.973913192749023, 'learning_rate': 9.670329670329671e-05, 'epoch': 0.04}


                                                
  4%|▎         | 10/273 [00:15<01:12,  3.64it/s] 

{'eval_loss': 1.3309156894683838, 'eval_accuracy': 0.33394160866737366, 'eval_runtime': 12.5508, 'eval_samples_per_second': 43.663, 'eval_steps_per_second': 10.916, 'epoch': 0.04}


  7%|▋         | 20/273 [00:21<01:54,  2.22it/s]

...

                                                 
 99%|█████████▉| 270/273 [08:32<00:01,  2.04it/s]

{'eval_loss': 0.5113503932952881, 'eval_accuracy': 0.6733576655387878, 'eval_runtime': 13.4921, 'eval_samples_per_second': 40.616, 'eval_steps_per_second': 10.154, 'epoch': 0.99}


100%|██████████| 273/273 [08:39<00:00,  1.90s/it]

{'train_runtime': 519.0099, 'train_samples_per_second': 4.216, 'train_steps_per_second': 0.526, 'train_loss': 0.7536402115455041, 'epoch': 1.0}


TrainOutput(global_step=273, training_loss=0.7536402115455041, metrics={'train_runtime': 519.0099, 'train_samples_per_second': 4.216, 'train_steps_per_second': 0.526, 'total_flos': 5.30858251542528e+16, 'train_loss': 0.7536402115455041, 'epoch': 0.9981718464351006})

In [33]:
# trainer.evaluate()
trainer.evaluate(eval_dataset=eval_dataset)

100%|██████████| 137/137 [00:13<00:00, 10.06it/s]


{'eval_loss': 0.5129837989807129,
 'eval_accuracy': 0.6715328693389893,
 'eval_runtime': 13.7399,
 'eval_samples_per_second': 39.884,
 'eval_steps_per_second': 9.971,
 'epoch': 0.9981718464351006}

In [34]:
trainer.evaluate(eval_dataset=train_dataset)

100%|██████████| 547/547 [00:55<00:00,  9.89it/s]


{'eval_loss': 0.5211272239685059,
 'eval_accuracy': 0.662705659866333,
 'eval_runtime': 55.4828,
 'eval_samples_per_second': 39.436,
 'eval_steps_per_second': 9.859,
 'epoch': 0.9981718464351006}